In [62]:
import json

import pandas as pd
# import numpy as np

from shapely import affinity
from shapely.geometry import mapping, shape

from bokeh.plotting import figure
from bokeh.palettes import Viridis6 as palette
from bokeh.io import (show,
                      output_notebook,
                      push_notebook,
                      output_file)
from bokeh.models import (GeoJSONDataSource,
                          CategoricalColorMapper,
                          HoverTool,
                          ColumnDataSource,
                          ColorBar,
                          LogColorMapper,
                          LogTicker,
                          FixedTicker,
                          Title,
                          OpenURL,
                          TapTool
                         )


In [ ]:
def get_geojson():

    def match_states(states_json):
        return states_json['properties']['NAME'] in tbl.index.tolist()

    cols = tbl.columns.tolist() 
    with open('data/homocide_wiki/us_states_simplified_albers.geojson') as us_states_file:
        geojson = json.loads(us_states_file.read())
        geojson['features'] = list(filter(match_states,geojson['features']))
        for f in geojson['features']:
            row = tbl.loc[f['properties']['NAME']]
            for col in cols:
                f['properties'][col] = row[col]
            if f['properties']['NAME'] == 'Alaska':
                geom = affinity.translate(shape(f['geometry']), xoff=1.2e6, yoff=-4.8e6)
                geom = affinity.scale(geom, .4, .4)
                geom = affinity.rotate(geom, 30)
                f['geometry'] = mapping(geom)
            if f['properties']['NAME'] == 'Hawaii':
                geom = affinity.translate(shape(f['geometry']), xoff=4.6e6, yoff=-1.1e6)
                geom = affinity.rotate(geom, 30)
                f['geometry'] = mapping(geom)
    return json.dumps(geojson) 

In [3]:
tbl = pd.read_csv('data/state_data.csv')
tbl.set_index('State',inplace=True)
tbl['AFFL'] = tbl['# Affiliate Members as of 2/12/2019'].astype(float)
tbl['STUD'] = tbl['# Student Members as of 2/12/2019'].astype(float)
tbl['CAFF'] = tbl['# CNM/CM Affiliate Members (excludes Student & Associate)'].astype(float)
tbl['AMCB'] = tbl['# CNM/Cms as of Aug 2018 (data from AMCB)']
tbl['PCT'] = tbl['% of CNM/CM Members out of Total CNM/CMs']
tbl[tbl.columns.tolist()[:3]] = tbl[tbl.columns.tolist()[:3]].astype(float)
tbl.head()

,# Affiliate Members as of 2/12/2019,# Student Members as of 2/12/2019,# CNM/CM Affiliate Members (excludes Student & Associate),# CNM/Cms as of Aug 2018 (data from AMCB),% of CNM/CM Members out of Total CNM/CMs,AFFL,STUD,CAFF,AMCB,PCT
State,,,,,,,,,,
Alabama,19,3,14,47,29.8%,19.0,3.0,14.0,47,29.8%
Alaska,60,10,50,110,45.5%,60.0,10.0,50.0,110,45.5%
Arizona,139,22,111,232,47.8%,139.0,22.0,111.0,232,47.8%
Arkansas,18,3,13,32,40.6%,18.0,3.0,13.0,32,40.6%
California,414,86,318,1112,28.6%,414.0,86.0,318.0,1112,28.6%


In [12]:
color_mapper = LogColorMapper(palette=palette)
year= '# CNM/CM Affiliate Members (excludes Student & Associate)'
fig = figure(title="Midwifery integration in the United States",plot_width=1250, plot_height=600, x_range=(-3e6, 3e6), y_range=(-2e6, 1.5e6))
fig.title.text_font_size = "25px"
fig.title.align = "center"
fig.grid.grid_line_alpha = 0
fig.axis.visible = False
fig.patches(xs='xs', ys='ys', 
            line_color='white', 
            color=dict(transform=color_mapper,field=year),
            source=GeoJSONDataSource(geojson=get_geojson()), 
            alpha=.85)


hover = HoverTool()

hover.tooltips = """    
        <div>
            <span style="font-size: 17px; font-weight: bold;">@NAME</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">Affiliate Members as of 2/12/2019</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">Student Members as of 2/12/2019</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">CNM/CM Affiliate Members (excludes Student & Associate)</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">CNM/Cms as of Aug 2018 (data from AMCB)</span>
        </div>
        <div>
            <span style="font-size: 15px; color: tomato;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">of CNM/CM Members out of Total CNM/CMs</span>
        </div>

    """.format('AFFL','STUD','CAFF','AMCB','PCT')

hover.point_policy = "follow_mouse"
fig.add_tools(hover)   
    
output_file('heart_of_touch.html')
show(fig)

In [18]:
tbl.CAFF.max()

380.0

In [69]:
color_mapper = LogColorMapper(palette=palette, low=tbl.CAFF.min(), high=tbl.CAFF.max())
column= 'CAFF'
fig = figure(title="Midwifery integration in the United States",plot_width=1250, plot_height=600, 
             x_range=(-3e6, 3e6), y_range=(-2e6, 1.5e6), 
#              tools="tap"
            )
fig.title.text_font_size = "25px"
fig.title.align = "center"
fig.grid.grid_line_alpha = 0
fig.axis.visible = False
fig.patches(xs='xs', ys='ys', 
            line_color='grey', 
            color=dict(transform=color_mapper,field=column),
            source=GeoJSONDataSource(geojson=get_geojson()), 
            alpha=.85)
color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0), ticker=FixedTicker(ticks=[1,5,10,50,100,200,300]))

fig.add_layout(color_bar, 'left')
fig.add_layout(Title(text="# of CNM/CM Affiliate Members (excludes Student & Associate)", align="center"), "left")
hover = HoverTool()

hover.tooltips = """    
        <div>
            <span style="font-size: 17px; font-weight: bold;">@NAME</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">Affiliate Members as of 2/12/2019</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">Student Members as of 2/12/2019</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">CNM/CM Affiliate Members (excludes Student & Associate)</span>
        </div>
        <div>
            <span style="font-size: 15px; color: orange;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">CNM/Cms as of Aug 2018 (data from AMCB)</span>
        </div>
        <div>
            <span style="font-size: 15px; color: tomato;"><b>@{}</b> :</span>
            <span style="font-size: 15px;">of CNM/CM Members out of Total CNM/CMs</span>
        </div>

    """.format('AFFL','STUD','CAFF','AMCB','PCT')

hover.point_policy = "follow_mouse"
fig.add_tools(hover)   

# url = "https://www.heartoftouchfilm.com/"
# taptool = fig.select(type=TapTool)
# taptool.callback = OpenURL(url=url)

output_file('heart_of_touch.html')
show(fig)



In [65]:
# AttributeError: unexpected attribute 'ticks' to LogTicker, possible attributes are 
#     base, desired_num_ticks, js_event_callbacks, js_property_callbacks, mantissas, max_interval, 
#     min_interval, name, num_minor_ticks, subscribed_events or tags

# ValueError: expected an element of either 
#     Enum('top_left', 'top_center', 'top_right', 'center_left', 'center', 'center_right', 'bottom_left', 
#          'bottom_center', 'bottom_right') or Tuple(Float, Float), got 'right'
